In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import sklearn.svm as svm
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV




In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

class save_images ( object ):
    def __init__( self ):
        self.i = 0
    def __call__( self, image):
        # do something
        self.i +=1
        cv2.imwrite("test_images2/image" + str(self.i) + ".jpg", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        return image
    
save_images = save_images()    
    
## creating additional test images
from moviepy.editor import VideoFileClip
output_video_name = 'test_video_output.mp4'
clip1 = VideoFileClip("test_video.mp4")
clip2 = clip1.subclip(0,2)
output_video = clip2.fl_image(save_images) #NOTE: this function expects color images!!
%time output_video.write_videofile(output_video_name, audio=False)
import os
os.remove('test_video_output.mp4')
print ("Completed")


## image_features_extractor

In [ ]:
class image_features_extractor:
    def __init__(self    \
        ,color_space = 'HSV'      # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
        ,orient = 9               # HOG orientations 
        ,pix_per_cell = 7         # HOG pixels per cell 
        ,cell_per_block = 2       # HOG cells per block 
        ,spatial_size = (32, 32)  # Spatial binning dimensions
        ,spatial_feat = True     # Spatial features on or off
        ,hist_feat = True        # Histogram features on or off
        ,hog_feat = True          # HOG features on or off
        ,nbins=32                 # Number of color histogram bins
        ,bins_range=(0, 256)
                ):

        self.color_space = color_space
        self.orient = orient
        self.pix_per_cell = pix_per_cell
        self.cell_per_block = cell_per_block
        self.spatial_size = spatial_size
        self.spatial_feat = spatial_feat
        self.hist_feat = hist_feat
        self.hog_feat = hog_feat
        self.nbins=nbins
        self.bins_range=bins_range
        

    # Define a function to compute binned color features  
    def bin_spatial(self,feature_image):
        # Use cv2.resize().ravel() to create the feature vector
        features = cv2.resize(feature_image, self.spatial_size).ravel() 
        # Return the feature vector
        return features

    # Define a function to compute color histogram features  
    def color_hist(self,feature_image):
        # Compute the histogram of the color channels separately
        channel1_hist = np.histogram(feature_image[:,:,0], bins=self.nbins, range=self.bins_range)
        channel2_hist = np.histogram(feature_image[:,:,1], bins=self.nbins, range=self.bins_range)
        channel3_hist = np.histogram(feature_image[:,:,2], bins=self.nbins, range=self.bins_range)
        # Concatenate the histograms into a single feature vector
        hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
        # Return the individual histograms, bin_centers and feature vector
        return hist_features

    def get_hog_features(self, feature_image, visualise=False):
        if visualise:
            # feature_image is 1 channel image
            hog_features, hog_image     = hog(feature_image, 
                                  orientations=self.orient, 
                                  pixels_per_cell=(self.pix_per_cell, self.pix_per_cell),
                                  cells_per_block=(self.cell_per_block, self.cell_per_block), 
                                  block_norm='L2-Hys',
                                  transform_sqrt=True, 
                                  visualise=True, feature_vector=False)
            return hog_features, hog_image

        
        else:
            # feature_image is 1 channel image
            hog_features     = hog(feature_image, 
                                  orientations=self.orient, 
                                  pixels_per_cell=(self.pix_per_cell, self.pix_per_cell),
                                  cells_per_block=(self.cell_per_block, self.cell_per_block), 
                                  block_norm='L2-Hys',
                                  transform_sqrt=True, 
                                  visualise=False, feature_vector=True)
            return hog_features



    def get_all_features (self, img):
        if self.color_space != 'RGB':
            if self.color_space == 'HSV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            elif self.color_space == 'LUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
            elif self.color_space == 'HLS':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
            elif self.color_space == 'YUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
            elif self.color_space == 'YCrCb':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else: 
            feature_image = np.copy(img)


        img_features = []

        if self.spatial_feat == True:
            img_features.append(self.bin_spatial(feature_image))

        if self.hist_feat == True:
            img_features.append(self.color_hist(feature_image))
            
        if self.hog_feat == True:
            hog_features = []
            for channel in range(feature_image.shape[2]):
                temp = self.get_hog_features(feature_image[:,:,channel])
                hog_features.extend(temp)      
            img_features.append(hog_features)

        #9) Return concatenated array of features
    #     print (img_features)
        return np.concatenate(img_features)                
    

### Car Detector from images ( train & prediction )

In [ ]:
class image_car_detector:
    def __init__(self, extractor, 
                 cars_folder='vehicles', 
                 notcars_folder='non-vehicles',
                 limit_images=None):
        self.cars_folder    = cars_folder
        self.notcars_folder = notcars_folder

        self.car_images_list     = []
        self.notcars_images_list = []
        self.extractor = extractor
        
        self.car_features = []
        self.notcar_features = []
        self.limit_images = limit_images

        
    def load_images_list(self):
        self.car_images_list     = []
        self.notcars_images_list = []
    
        # loading cars
        for subfolder in glob.glob(self.cars_folder + '/*'):
          print ("subfolder = " + subfolder)
          for image  in  glob.glob(subfolder+'/*'):
               self.car_images_list.append(image)

        # loading non cars
        for subfolder in glob.glob(self.notcars_folder+ '/*'):
          for image  in  glob.glob(subfolder+'/*'):
               self.notcars_images_list.append(image)
    
    # Define a function to extract features from a list of images
    # Have this function call bin_spatial() and color_hist()
    def extract_imgs_features(self):
        # Iterate through the list of images
        if self.limit_images:
            x_limit = self.limit_images
            print (" limiting the number of images to " + str(x_limit))
        else:
            x_limit = len(self.car_images_list)
            print (" processing " + str(x_limit) +" images "  )


        print ( " ")
        print (" Extracting features from cars ....")
        # CARS FEATURES 
        self.car_features = []


        for file in self.car_images_list[:x_limit]:
            image = cv2.imread(file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            self.car_features.append(self.extractor.get_all_features(image))
        print (" .. completed ")

            
        print ( " ")
        print (" Extracting features from non cars ....")
        # NOT CAR FEATURES 
        self.notcar_features = []
        # Iterate through the list of images
        for file in self.notcars_images_list[:x_limit]:
            image = cv2.imread(file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            self.notcar_features.append(self.extractor.get_all_features(image))
        print (" .. completed ")
        print (" ")

        
    def detector_train(self, parameters=[1, 10]):
        print ( "Starting the training ....")
        if len(self.car_features) == 0:
            print ( " ... extracting the features ")
            self.extract_imgs_features()
        
        X = np.vstack((self.car_features, self.notcar_features)).astype(np.float64)                        

        print (".....Scaling...")
        # Fit a per-column scaler
        self.X_scaler = StandardScaler().fit(X)
        # Apply the scaler to X
        scaled_X = self.X_scaler.transform(X)
        
        y = np.hstack((np.ones(len(self.car_features)), np.zeros(len(self.notcar_features))))

        from sklearn.model_selection import train_test_split

        # Split up data into randomized training and test sets
        rand_state = np.random.randint(0, 100)
        X_train, X_test, y_train, y_test = train_test_split(
            scaled_X, y, test_size=0.2, random_state=rand_state)
        
        from sklearn.svm import LinearSVC
        import time

#         self.svr = svm.LinearSVC()


        # Check the training time for the SVC
        print ( "Start training ... ")
        t=time.time()
#         param_grid = {'C': parameters}
#         self.clf = GridSearchCV(self.svr , param_grid=param_grid)
        self.clf = LinearSVC()
        self.clf.fit(X_train, y_train)

        t2 = time.time()
        print(round(t2-t, 2), 'Seconds to train SVC...')

        # Check the score of the SVC
        print('Test Accuracy of SVC = ', round(self.clf.score(X_test, y_test), 4))

    def save_classifier(self):
        import pickle
        # save the classifier
        with open('car_detection_classifier.pkl', 'wb') as fid:
            pickle.dump(self.clf, fid) 
            
        with open('X_scaler.pkl', 'wb') as fid:
            pickle.dump(self.X_scaler, fid) 
        print (" .. classifier saved ")    

    def load_classifier(self):
        import pickle
        # save the classifier
        with open('car_detection_classifier.pkl', 'rb') as fid:
            self.clf = pickle.load(fid)
        with open('X_scaler.pkl', 'rb') as fid:
            self.X_scaler = pickle.load(fid)

        print (" .. classifier loaded ")    


    def search_windows(self, img, windows):
        self.on_windows = []
        for window in windows:
#             print (window)
            #3) Extract the test window from original image
            test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
            #4) Extract features for that window using single_img_features()
            features = self.extractor.get_all_features(test_img)
            #5) Scale extracted features to be fed to classifier
            test_features = self.X_scaler.transform(np.array(features).reshape(1, -1))
            
            #6) Predict using your classifier
            prediction = self.clf.predict(test_features)
            #7) If positive (prediction == 1) then save the window
            if prediction == 1:
                self.on_windows.append(window)
#                 print ( "prediction = 1 ")
#                 plt.title("prediction = 1 ")
#             else:
#                 plt.title("prediction = 0 ")

#             plt.axis('off')
#             plt.imshow(test_img)
#             plt.show()


                
        return self.on_windows

            

In [ ]:
import cv2
image = cv2.imread('vehicles/GTI_Far/image0022.png')
# image = cv2.imread('test_images/test1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# color space = RGB, HSV, LUV, HLS, YUV, YCrCb
extractor = image_features_extractor(color_space='YCrCb',
                                     pix_per_cell=8,
                                     cell_per_block=2,
                                     
                                     orient=9)
hog_features, image_tmp = extractor.get_hog_features(image[:,:,2], visualise=True)
plt.imshow(image)
plt.show()
plt.imshow(image_tmp,cmap='gray')
plt.show()
print ( len(hog_features))
print(np.array(hog_features).shape)

In [ ]:
extractor = image_features_extractor(color_space='YCrCb'
                                    ,pix_per_cell=8
                                    ,cell_per_block=2
                                     
                                    ,orient=9
                                    ,spatial_feat = True     # Spatial features on or off
                                    ,hist_feat = True        # Histogram features on or off
                                    ,hog_feat = True  
                                    ,spatial_size = (16, 16)
                                    )



detector = image_car_detector(extractor) #, limit_images= 2000)

# detector.load_images_list()

# detector.extract_imgs_features()



In [ ]:
len(detector.car_features[0])

In [ ]:
# detector.detector_train()

# detector.save_classifier()

detector.load_classifier()

In [ ]:
class window_extractor:
    def __init__(self, image,               \
                 x_start_stop=[None, None], \
                 y_start_stop=[350, None], \
                 window_dimension=64,  \
                 x_overlap=0.5, y_overlap=0.5):
        
        self.image           = image
        self.xy_window       = (window_dimension, window_dimension)
        self.xy_overlap      = (x_overlap, y_overlap)
        self.x_start_stop    = x_start_stop
        self.y_start_stop    = y_start_stop
        
        # If x and/or y start/stop positions not defined, set to image size
        #     print (x_start_stop)
        if self.x_start_stop[0] == None:
            self.x_start_stop[0] = 0
        if self.x_start_stop[1] == None:
            self.x_start_stop[1] = self.image.shape[1]
        if self.y_start_stop[0] == None:
            self.y_start_stop[0] = 0
        if self.y_start_stop[1] == None:
            self.y_start_stop[1] = self.image.shape[0]

        # Compute the span of the region to be searched    
        self.xspan = self.x_start_stop[1] - self.x_start_stop[0]
        self.yspan = self.y_start_stop[1] - self.y_start_stop[0]
        
        # Compute the number of pixels per step in x/y
        self.nx_pix_per_step = np.int(self.xy_window[0]*(1 - self.xy_overlap[0]))
        self.ny_pix_per_step = np.int(self.xy_window[1]*(1 - self.xy_overlap[1]))
        
        # Compute the number of windows in x/y
        self.nx_buffer = np.int(self.xy_window[0]*(self.xy_overlap[0]))
        self.ny_buffer = np.int(self.xy_window[1]*(self.xy_overlap[1]))
        self.nx_windows = np.int((self.xspan-self.nx_buffer)/self.nx_pix_per_step) 
        self.ny_windows = np.int((self.yspan-self.ny_buffer)/self.ny_pix_per_step) 



    def get_windows(self):
        # Initialize a list to append window positions to
        self.window_list = []
        # Loop through finding x and y window positions
        # Note: you could vectorize this step, but in practice
        # you'll be considering windows one by one with your
        # classifier, so looping makes sense
        for ys in range(self.ny_windows):
            for xs in range(self.nx_windows):
                # Calculate window position
                startx = xs*self.nx_pix_per_step + self.x_start_stop[0]
                endx = startx + self.xy_window[0]
                starty = ys*self.ny_pix_per_step + self.y_start_stop[0]
                endy = starty + self.xy_window[1]
                # Append window position to list
                self.window_list.append(((startx, starty), (endx, endy)))
        # Return the list of windows
        return self.window_list
    
    def draw_boxes(self,hot_windows, color=(0, 0, 255), thick=6):
        # Make a copy of the image
        imcopy = np.copy(self.image)
        tmp_window_list = hot_windows
        for bbox in tmp_window_list:
#             print ("bbox = " + str(bbox))
            # Draw a rectangle given bbox coordinates
            cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
        # Return the image copy with boxes drawn
        return imcopy

    

In [ ]:
import cv2
image = mpimg.imread('test_images/test3.jpg')

windows = []

##########  scale = 1
windows_tmp = window_extractor(image, 
                               window_dimension=int(64), 
                               x_overlap = 0.7,
                               y_overlap = 0.7,
                               x_start_stop = [340,None],
                               y_start_stop = [400,496]).get_windows()
for x in windows_tmp:
     windows.append(x)
        
# ###########  scale = 1.25      
# windows_tmp = window_extractor(image, 
#                                window_dimension=int(64*1.25), 
#                                y_overlap = 0.5,
#                                y_start_stop = [400,550]).get_windows()
# for x in windows_tmp:
#      windows.append(x)

# ###########  scale = 1.5     
# windows_tmp = window_extractor(image, 
#                                window_dimension=int(64*1.5), 
#                                y_overlap = 0.5,x_overlap=0.6,
#                                y_start_stop = [400,592]).get_windows()
# for x in windows_tmp:
#      windows.append(x)

# ###########  scale = 1.75     
# windows_tmp = window_extractor(image, 
#                                window_dimension=int(64*1.9), 
#                                y_overlap = 0.5,x_overlap=0.6,
#                                y_start_stop = [400,656]).get_windows()
# for x in windows_tmp:
#      windows.append(x)


        
hot_windows = detector.search_windows( image, windows)

print (" all boxes")
plt.imshow(window_extractor(image).draw_boxes(windows))
plt.show()


print (" recognized cars boxes")
plt.imshow(window_extractor(image).draw_boxes(hot_windows))
plt.show()





In [ ]:
from scipy.ndimage.measurements import label

class heat_map:
    def __init__(self, boxes_list, image):
            self.boxes = boxes_list
            self.heatmap = np.zeros_like(image[:,:,0]).astype(np.float)
            self.image = np.copy(image)
            
    
    def add_heat(self):
        # Iterate through list of bboxes
        
        for box in self.boxes :
#             print ("box = " + str(box))
            # Assuming each "box" takes the form ((x1, y1), (x2, y2))
            self.heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

        # Return updated heatmap

    def apply_threshold(self,threshold):
        print (" Applying threshold = " + str(threshold))
        # Zero out pixels below the threshold
        self.heatmap[self.heatmap < threshold] = 0
        # Return thresholded map

    def draw_labeled_bboxes(self):
        print ("draw_labeled_bboxes .... ")
        #scipy.ndimage.measurements.label
        self.labels = label(self.heatmap)
#         print ("labels = " + str(labels))
        # Iterate through all detected cars
        self.boxes = []
        for car_number in range(1, self.labels[1]+1):
            print ( " car_number = " + str(car_number))
            # Find pixels with each car_number label value
            nonzero = (self.labels[0] == car_number).nonzero()
            # Identify x and y values of those pixels
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            # Define a bounding box based on min/max x and y
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            # Draw the box on the image
            cv2.rectangle(self.image, bbox[0], bbox[1], (0,255,0), 6)
            self.boxes.append(bbox)
        # Return the image


    

In [ ]:
heat = heat_map(hot_windows, image)
heat.add_heat()
heat.apply_threshold(threshold=1)
heat.draw_labeled_bboxes()

# print(heat.heatmap.nonzero())

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()

ax1.imshow(image)
ax1.set_title('Original Image', fontsize=50)
# Turn off tick labels
ax1.set_yticklabels([])
ax1.set_xticklabels([])

ax2.imshow(heat.image)
ax2.set_title('Identified cars', fontsize=50)
# Turn off tick labels
ax2.set_yticklabels([])
ax2.set_xticklabels([])


plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

plt.show()



### Now i am trying to refine the boxes found putting more sliding windows around of the detected vehicles, 
### to better include the car in a correct box

In [ ]:
heat.boxes

for box in heat.boxes:
    print (box)
    print (box[0][1])
    print (box[1][1])

In [ ]:
windows_new = []

for box in heat.boxes:
    x_start = int((box[0][0] ) * .95)
    x_stop  = int((box[1][0] ) * 1.05)
    y_start = int((box[0][1] ) * .95)
    y_stop  = int((box[1][1] ) * 1.05)
    
    ##########  scale = 1
    windows_tmp_new = window_extractor(image, 
                                   window_dimension=64,
                                   x_overlap = 0.9,
                                   y_overlap = 0.9,
                                   x_start_stop = [x_start,x_stop],
                                   y_start_stop = [y_start,y_stop]).get_windows()
    for x in windows_tmp_new:
         windows_new.append(x)
            
hot_windows_new = detector.search_windows( image, windows_new)

print (" all boxes")
plt.imshow(window_extractor(image).draw_boxes(windows_new))
plt.show()


print (" recognized cars boxes")
plt.imshow(window_extractor(image).draw_boxes(hot_windows_new))
plt.show()

In [ ]:
heat_new = heat_map(hot_windows_new, image)
heat_new.add_heat()
heat_new.apply_threshold(threshold=3)
heat_new.draw_labeled_bboxes()

# print(heat.heatmap.nonzero())

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()

ax1.imshow(image)
ax1.set_title('Original Image', fontsize=50)
# Turn off tick labels
ax1.set_yticklabels([])
ax1.set_xticklabels([])

ax2.imshow(heat_new.image)
ax2.set_title('Identified cars', fontsize=50)
# Turn off tick labels
ax2.set_yticklabels([])
ax2.set_xticklabels([])


plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

plt.show()




In [ ]:


heat.draw_labeled_bboxes()
plt.imshow(heat.image)
plt.show()

In [ ]:
len(windows)

### Defining the class pipeline

In [ ]:
class pipeline:
    def __init__(self, debug=False):
        self.detector = image_car_detector(image_features_extractor
                                               (color_space='YCrCb'
                                                ,pix_per_cell=8
                                                ,cell_per_block=2
                                                ,orient=9
                                                ,spatial_feat = True     # Spatial features on or off
                                                ,hist_feat = True        # Histogram features on or off
                                                ,hog_feat = True  
                                                ,spatial_size = (16, 16)
                                            )
                                           )
        self.detector.load_classifier()
        self.debug= debug
        
    def set_image(self, image):
            self.image = image
            
    def searchfor_windows_first(self):
            self.windows = []
            ##########  scale = 1
            windows_ext = window_extractor(self.image, 
                                           window_dimension=80, 
                                           x_overlap = 0.3,
                                           y_overlap = 0.3,
                                           x_start_stop = [340,None],
                                           y_start_stop = [400,496])
            
            for x in windows_ext.get_windows():
                 self.windows.append(x)

            ##########  scale = 1.25      
            windows_ext = window_extractor(image, 
                                           window_dimension=int(80*1.25), 
                                           x_overlap = 0.3,
                                           y_overlap = 0.3,
                                           x_start_stop = [340,None],
                                           y_start_stop = [400,550])
            for x in windows_ext.get_windows():
                 self.windows.append(x)


            # ###########  scale = 1.5     
            windows_ext = window_extractor(image, 
                                           window_dimension=int(80*1.5), 
                                           x_overlap = 0.3,
                                           y_overlap = 0.3,
                                           x_start_stop = [340,None],
                                           y_start_stop = [400,600])
            for x in windows_ext.get_windows():
                 self.windows.append(x)

        
            # ###########  scale = 1.75   
            windows_ext = window_extractor(image, 
                                           window_dimension=int(80*1.75), 
                                           x_overlap = 0.3,
                                           y_overlap = 0.3,
                                           x_start_stop = [340,None],
                                           y_start_stop = [400,656])
            for x in windows_ext.get_windows():
                 self.windows.append(x)





    def searchfor_windows_second(self, 
                                 x_overlap=None,
                                 y_overlap=None,
                                 x_start_stop=None,
                                 y_start_stop=None
                                ):
            windows_ext = window_extractor(self.image, 
                                           window_dimension=int(64), 
                                           x_overlap = 0.9,
                                           y_overlap = 0.9,
                                           x_start_stop=x_start_stop,
                                           y_start_stop=y_start_stop)
            
            for x in windows_ext.get_windows():
                 self.windows.append(x)
        
            if self.debug:
                image_tmp = windows_ext.draw_boxes(self.windows)
                return image_tmp
            else:
                return None



    # search with different steps
    # first search is with big windows and low number of windows to get a speed overview of the car positions
    # secondly using the car position I will "shut" more smaller sliding windows in that area to find the 
    #          exact box around the car
    def search_cars_by_steps(self):
        
        ######################################################
        ##              FIRST         STEP
        ######################################################

        self.windows = []
        self.hot_windows = []
        
        ########
        # creating a list of boxes
        ########
        tmp_image = self.searchfor_windows_first()
        if self.debug:
            image_tmp = window_extractor(self.image).draw_boxes(self.windows)
            plt.imshow(image_tmp)    
            plt.title(" Boxes drawn on the image ")
            plt.show()

            

        ########
        # search for car detection in the above calculated boxes
        ########
        self.hot_windows = []
        self.hot_windows = self.detector.search_windows(self.image, self.windows)
        if self.debug:
            tmp_image_hot = window_extractor(self.image).draw_boxes(self.hot_windows)
            plt.imshow(tmp_image_hot)    
            plt.title(" Car boxes detected ")
            plt.show()

            
        ########
        # heat map
        ########
        self.heat = heat_map(self.hot_windows, self.image)
        self.heat.add_heat()
        self.heat.apply_threshold(threshold=1)
#         self.heat.draw_labeled_bboxes()

        if self.debug:
            self.heat.draw_labeled_bboxes()
            plt.imshow(self.heat.image)    
            plt.title(" First heat map")

            plt.show()


# #         return tmp_image, tmp_image_hot, self.heat.image


        ######################################################
        ##              SECOND         STEP
        ######################################################


        if self.debug: print ( " Second Step ....")
        self.windows = []
        for box in self.heat.boxes:
            x_start = int((box[0][0] ) * .95)
            x_stop  = int((box[1][0] ) * 1.05)
            y_start = int((box[0][1] ) * .95)
            y_stop  = int((box[1][1] ) * 1.05)

            print (" box = " + "x_start= " + str(x_start) +  " , "  
                             + "x_stop= "  + str(x_stop)  +  " , "  
                             + "y_start= " + str(y_start) +  " , "  
                             + "y_stop= "  + str(y_stop)  +  " , "  
                  )
            # creating a list of boxes
            self.searchfor_windows_second(
                                           x_overlap = 0.7,
                                           y_overlap = 0.7,
                                           x_start_stop=[x_start,x_stop],
                                           y_start_stop=[y_start,y_stop]
                                )

        if self.debug:
            image_tmp = window_extractor(self.image).draw_boxes(self.windows)
            plt.imshow(image_tmp)    
            plt.title(" Boxes drawn on the image (second step )")
            plt.show()

            
        # search for car detection in the above calculated boxes
        if self.debug:print (" Searching for windows with cars ")
        if self.debug:print ( " .. Number of boxes = " + str(len(self.windows)) )
        self.hot_windows = []
        self.hot_windows = self.detector.search_windows(self.image, self.windows)
        if self.debug:
            tmp_image_hot = window_extractor(self.image).draw_boxes(self.hot_windows)
            plt.imshow(tmp_image_hot)    
            plt.title(" Car boxes detected ")
            plt.show()

        ########
        # heat map
        ########
        if self.debug: print ("Creating heat_map ... ")
        self.heat = heat_map(self.hot_windows, self.image)
        self.heat.add_heat()
        self.heat.apply_threshold(threshold=5)
        
        if self.debug:
            self.heat.draw_labeled_bboxes()
            plt.imshow(self.heat.image)    
            plt.title(" Second heat map")

            plt.show()



        if self.debug: print(".... Completed")



In [ ]:
pipeline1 = pipeline(debug=True)

In [ ]:
import cv2
image = mpimg.imread('test_images2/image1.jpg')

pipeline1.set_image(image)
pipeline1.search_cars_by_steps()

